<a href="https://colab.research.google.com/github/Altynny/EfficientNet_SRH_6000Mhz/blob/main/notebooks/ds_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Загрузка датасета

Загрузить данные с сервера

In [ ]:
!wget https://forecasting.iszf.irk.ru/data/6000/all.zip

In [ ]:
!mkdir -p data/6000Mhz
!unzip -qqd data/6000Mhz all.zip

Или с диска

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!mkdir -p data/6000Mhz
!unzip -qqd data/6000Mhz /content/drive/MyDrive/SRH_dataset/all.zip

# Подготовка датасета в соответсвие с классами в CSV

Сначала считываем CSV и корректируем имена файлов с изображениями

In [ ]:
import pandas as pd

data = pd.read_csv('dataset_6000.csv')
data.Name = data.Name.apply(lambda x: x.replace(':', '-') + '.png')
data.head()

,Name,Date,Time,URL,Category,Probability,Color
0,srh_I_2022-03-17T02-59-30_6200.fit.png,2022-03-17,02:59:30,https://ftp.rao.istp.ac.ru/SRH/SRH0612/cleanMa...,Ok,1,red
1,srh_I_2022-03-17T03-11-32_6200.fit.png,2022-03-17,03:11:32,https://ftp.rao.istp.ac.ru/SRH/SRH0612/cleanMa...,Ok,1,red
2,srh_I_2022-03-17T02-32-10_6200.fit.png,2022-03-17,02:32:10,https://ftp.rao.istp.ac.ru/SRH/SRH0612/cleanMa...,Ok,1,red
3,srh_I_2022-03-17T02-17-57_6200.fit.png,2022-03-17,02:17:57,https://ftp.rao.istp.ac.ru/SRH/SRH0612/cleanMa...,Ok,1,red
4,srh_I_2022-03-17T03-20-16_6200.fit.png,2022-03-17,03:20:16,https://ftp.rao.istp.ac.ru/SRH/SRH0612/cleanMa...,Bad,1,green


Часть данных оказалась повреждена, поэтому фильтруем датасет

In [ ]:
from PIL import Image

DATA_DIR = 'data/6000Mhz'

def check_corruption(filename, base_dir = DATA_DIR):
  try:
    with Image.open(f'{base_dir}/{filename}') as img:
      img.verify()
      return False
  except:
      return True

corrupted_mask = data.Name.apply(check_corruption)
corrupted_data = data[corrupted_mask == True]
filtred_data = data[corrupted_mask == False]

In [ ]:
corr_bad = corrupted_data[corrupted_data.Category == 'Bad'].shape[0]
corr_ok = corrupted_data[corrupted_data.Category == 'Ok'].shape[0]
print(f'Всего повреждено {corr_bad} файлов Bad и {corr_ok} файлов Ok')

Всего повреждено 784 файлов Bad и 113 файлов Ok


Теперь можно обрезать классы по величине минимального, чтобы избежать дисбаланса

In [ ]:
ok_names = filtred_data.Name[data.Category == 'Ok'].to_list()
bad_names = filtred_data.Name[data.Category == 'Bad'].to_list()

min_category_len = min(len(ok_names), len(bad_names))
print(f'Минимальный размер класса - {min_category_len}')

X = ok_names[:min_category_len] + bad_names[:min_category_len]
Y = ['Ok']*min_category_len + ['Bad']*min_category_len

Минимальный размер класса - 4891


Делим датасет на тренировочную, валидационную и тестовую выборки применив дважды train_test_split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.4) # сначала выделяем 40% на тестовую часть
X_val, X_test, Y_val, Y_test = train_test_split(X_test, Y_test, test_size = 0.5) # и выделяем половину из тестовой на валидационную

И перемещаем файлы распакованного архива в соответствие с размеченной структурой

In [ ]:
import os
from shutil import move

for ds_type, names, categories in [('train', X_train, Y_train), ('val', X_val, Y_val), ('test', X_test, Y_test)]:
  os.makedirs(f'{DATA_DIR}/{ds_type}/Ok', exist_ok=True)
  os.makedirs(f'{DATA_DIR}/{ds_type}/Bad', exist_ok=True)
  for name, category in zip(names, categories):
    source = f'{DATA_DIR}/{name}'
    if os.path.exists(source):
      move(source, f'{DATA_DIR}/{ds_type}/{category}/{name}')

# Очистка директории датасета от лишних файлов
for name in os.listdir(DATA_DIR):
  source = f'{DATA_DIR}/{name}'
  if os.path.isfile(source):
    os.remove(source)

In [ ]:
!zip -qqr drive/MyDrive/SRH_dataset/data.zip data